# Evaluation

In [7]:
import os

from keras import metrics

from gossiplearning.config import Config

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [8]:
import json

from utils.metrics import SimulationMetrics, plot_metrics, average_metrics, plot_metrics_boxplot, \
    dump_experiment_metrics
from pathlib import Path
from utils.evaluation import evaluate_simulations

In [9]:
with open("config.json", "r") as f:
    config = Config.model_validate(json.load(f))

In [10]:
dataset_base_dir = Path("data/datasets/1func_10nodes_3k")

In [11]:
simulations_metrics, simulations_generalization_metrics = evaluate_simulations(5, config,  dataset_base_dir, evaluate_generalization=True) 

Metal device set to: Apple M1 Pro
Metal device set to: Apple M1 Pro
Metal device set to: Apple M1 Pro
Metal device set to: Apple M1 Pro
Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB


systemMemory: 32.00 GB
maxCacheSize: 10.67 GB


systemMemory: 32.00 GB
maxCacheSize: 10.67 GB


systemMemory: 32.00 GB
maxCacheSize: 10.67 GB


systemMemory: 32.00 GB
maxCacheSize: 10.67 GB


In [12]:
averaged_metrics = SimulationMetrics(
    gossip=average_metrics([m.gossip for m in simulations_metrics]),
    single_training=average_metrics([m.single_training for m in simulations_metrics]),
    centralized=average_metrics([m.centralized for m in simulations_metrics]),
)

averaged_generalization_metrics = SimulationMetrics(
    gossip=average_metrics([m.gossip for m in simulations_generalization_metrics]),
    single_training=average_metrics([m.single_training for m in simulations_generalization_metrics]),
    centralized=average_metrics([m.centralized for m in simulations_generalization_metrics]),
)

aggregated_plots = Path(config.workspace_dir) / "plots"
aggregated_plots.mkdir(exist_ok=True, parents=True)

plot_metrics(averaged_metrics, aggregated_plots / "metrics_hist.png")
plot_metrics(averaged_generalization_metrics, aggregated_plots / "generalized_metrics_hist.png")

plot_metrics_boxplot(simulations_metrics, aggregated_plots / "metrics_boxplot.png")
plot_metrics_boxplot(simulations_generalization_metrics, aggregated_plots / "generalized_metrics_boxplot.png")

dump_experiment_metrics(averaged_metrics, Path(config.workspace_dir) / "metrics.csv")
dump_experiment_metrics(averaged_generalization_metrics, Path(config.workspace_dir) / "metrics_generalized.csv")

# test and prototyping

In [13]:
import numpy as np
dataset = np.load("data/datasets/1func_10nodes_3k/0/4in/node_0.npz")

In [14]:
X_test, Y_test = dataset["X_test"], dataset["Y_test"]

In [15]:
from keras.models import load_model
model = load_model("experiments/1fn_10n3k_0/0/models/0.h5")

In [16]:
p = model.predict(X_test)

327/327 [==============================] - 2s 4ms/step


In [17]:
Y_test.shape

(10452, 1)

In [ ]:

from utils.metrics import Metrics


def compute_metrics(truth: Union[np.ndarray, list[np.ndarray]], predicted: np.ndarray) -> Metrics:
    flattened_truth = truth.flatten()
    flattened_pred = predicted.flatten()

    return Metrics(
        mse=metrics.mean_squared_error(flattened_truth, flattened_pred),
        rmse=math.sqrt(metrics.mean_squared_error(flattened_truth, flattened_pred)),
        msle=metrics.mean_squared_log_error(flattened_truth, flattened_pred),
        mae=metrics.mean_absolute_error(flattened_truth, flattened_pred),
        mape=metrics.mean_absolute_percentage_error(flattened_truth, flattened_pred),
    )

In [6]:
import numpy as np
flattened_pred = np.concatenate(p)
flattened_pred

NameError: name 'p' is not defined

In [7]:
import math

n_elements = math.prod(Y_test.shape)

# x = Y_test.reshape((15456, ), order="F")
n_elements

NameError: name 'Y_test' is not defined

In [ ]:
from gossiplearning import History

history = History.model_validate(
        json.load((Path("experiments/3fn_10n3k_0/0") / "history.json").open("r"))
    )

In [ ]:
history.nodes_training_history[0].keys()

In [ ]:
from utils.evaluation import plot_node_history

plot_node_history(history.nodes_training_history[0], "test.png", fn_name="fn_0")